In [2]:
# !wget https://huggingface.co/datasets/mesolitica/instructions-dataset/resolve/main/shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('mesolitica/mallam-1.1B-4096')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
tokenizer.padding_side = "right"
tokenizer.chat_template = "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

In [2]:
def generate_and_tokenize_prompt(row):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    return {'text': prompt}

In [18]:
!rm -rf packing-mallam

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
import json

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

columns = {
    'input_ids': 'uint16',
}
hashes = 'sha1', 'xxh64'

block_size = 20480
temp = []
with MDSWriter(out='packing-mallam', columns=columns, compression=None, hashes=hashes) as out:
    with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
        for l in tqdm(fopen):
            row = json.loads(l)
            element = generate_and_tokenize_prompt(row)
            outputs = tokenizer(element['text'])
            temp.extend(outputs['input_ids'])
            done = False
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    out.write({
                        'input_ids': np.array(block).astype(np.uint16)
                    })
                

1617307it [35:23, 771.65it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2805667it [1:01:26, 802.57it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

3408182it [1:14:41, 760.49it/s]


In [20]:
dataset = LocalDataset('packing-mallam')

In [22]:
len(dataset) * block_size

2355814400

In [26]:
len(dataset)

115030

In [24]:
dataset[0]['input_ids']

array([   1, 1588,  395, ..., 5436,  323, 2507], dtype=uint16)

In [27]:
tokenizer.decode(dataset[1]['input_ids'])

' langkah-langkah yang dianjurkan oleh mereka untuk meningkatkan keberkesanan sistem pencegahan rasuah di Malaysia.\n\n5. Meningkatkan kesedaran awam - Kerajaan perlu melancarkan kempen kesedaran awam yang berterusan mengenai kesan buruk rasuah dan kepentingan membanterasnya. Ini boleh dilakukan melalui penerbitan bahan-bahan pendidikan, program pendidikan dan kempen media sosial.\n\nMelalui langkah-langkah ini, diharapkan kerajaan Malaysia dapat memperbaiki persepsi mengenai rasuah dan meningkatkan kepercayaan awam dalam usaha mereka membanteras gejala rasuah.</s> [INST] Menurut pendapat saya, kerajaan perlu melibatkan lebih banyak penyiasatan bebas dan bukan hanya bergantung kepada Suruhanjaya Pencegahan Rasuah Malaysia (SPRM) sahaja. Segala bentuk rasuah perlu diambil tindakan serius supaya rakyat dapat melihat usaha tegas untuk membanterasnya. Apa pendapat Tuan Menteri? [/INST]Saya bersetuju dengan pandangan yang dikemukakan. Kerajaan perlu melibatkan lebih banyak entiti penyiasata